In [4]:
import mwclient
import time

site = mwclient.Site("en.wikipedia.org")
page = site.pages["Bitcoin"]

In [5]:
revs = list(page.revisions())

In [6]:
revs[0]

OrderedDict([('revid', 1217143792),
             ('parentid', 1217051153),
             ('user', 'Wikideas1'),
             ('timestamp',
              time.struct_time(tm_year=2024, tm_mon=4, tm_mday=4, tm_hour=2, tm_min=33, tm_sec=8, tm_wday=3, tm_yday=95, tm_isdst=-1)),
             ('comment', '/* See also */ List of cryptocurrencies')])

In [7]:
revs = sorted(revs, key=lambda rev: rev["timestamp"])

In [8]:
revs[0]

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

In [11]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def classify_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [41]:
edits = {}

for rev in revs:
    date = time.strftime("%Y-%m-%d", rev["timestamp"])

    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)

    edits[date]["edit_count"] += 1
    comment = rev.get("comment","")
    edits[date]["sentiments"].append(classify_sentiment(comment))

In [42]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
    del edits[key]["sentiments"]

In [43]:
edits

{'2009-03-08': {'edit_count': 4,
  'sentiment': -0.5505250245332718,
  'neg_sentiment': 0.75},
 '2009-08-05': {'edit_count': 1,
  'sentiment': 0.748120903968811,
  'neg_sentiment': 0.0},
 '2009-08-06': {'edit_count': 2,
  'sentiment': 0.9957457184791565,
  'neg_sentiment': 0.0},
 '2009-08-14': {'edit_count': 1,
  'sentiment': 0.930020809173584,
  'neg_sentiment': 0.0},
 '2009-10-13': {'edit_count': 2,
  'sentiment': -0.22750070691108704,
  'neg_sentiment': 0.5},
 '2009-11-18': {'edit_count': 1,
  'sentiment': 0.8839504718780518,
  'neg_sentiment': 0.0},
 '2009-12-08': {'edit_count': 1,
  'sentiment': -0.9869275689125061,
  'neg_sentiment': 1.0},
 '2009-12-17': {'edit_count': 1,
  'sentiment': -0.9975171089172363,
  'neg_sentiment': 1.0},
 '2010-02-23': {'edit_count': 1,
  'sentiment': -0.9994946718215942,
  'neg_sentiment': 1.0},
 '2010-03-18': {'edit_count': 1,
  'sentiment': 0.875877857208252,
  'neg_sentiment': 0.0},
 '2010-04-13': {'edit_count': 4,
  'sentiment': 0.8443555235862732

In [44]:
import pandas as pd
edits_dataframe = pd.DataFrame.from_dict(edits, orient="index")

In [45]:
edits_dataframe

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.75
2009-08-05,1,0.748121,0.00
2009-08-06,2,0.995746,0.00
2009-08-14,1,0.930021,0.00
2009-10-13,2,-0.227501,0.50
...,...,...,...
2024-03-06,1,-0.961399,1.00
2024-03-17,4,-0.168054,0.50
2024-03-25,1,-0.998504,1.00
2024-04-03,2,-0.001680,0.50


In [46]:
edits_dataframe.index = pd.to_datetime(edits_dataframe.index)

In [47]:
from datetime import datetime

dates = pd.date_range(start="2009-03-08", end=datetime.today())

In [48]:
dates

DatetimeIndex(['2009-03-08', '2009-03-09', '2009-03-10', '2009-03-11',
               '2009-03-12', '2009-03-13', '2009-03-14', '2009-03-15',
               '2009-03-16', '2009-03-17',
               ...
               '2024-03-26', '2024-03-27', '2024-03-28', '2024-03-29',
               '2024-03-30', '2024-03-31', '2024-04-01', '2024-04-02',
               '2024-04-03', '2024-04-04'],
              dtype='datetime64[ns]', length=5507, freq='D')

In [49]:
edits_dataframe = edits_dataframe.reindex(dates, fill_value=0)

In [50]:
edits_dataframe

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.75
2009-03-09,0,0.000000,0.00
2009-03-10,0,0.000000,0.00
2009-03-11,0,0.000000,0.00
2009-03-12,0,0.000000,0.00
...,...,...,...
2024-03-31,0,0.000000,0.00
2024-04-01,0,0.000000,0.00
2024-04-02,0,0.000000,0.00
2024-04-03,2,-0.001680,0.50


In [51]:
rolling_edits = edits_dataframe.rolling(30).mean()

In [52]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-03-08,NaN,NaN,NaN
2009-03-09,NaN,NaN,NaN
2009-03-10,NaN,NaN,NaN
2009-03-11,NaN,NaN,NaN
2009-03-12,NaN,NaN,NaN
...,...,...,...
2024-03-31,0.200000,-0.070932,0.083333
2024-04-01,0.200000,-0.070932,0.083333
2024-04-02,0.200000,-0.070932,0.083333
2024-04-03,0.266667,-0.070988,0.100000


In [53]:
rolling_edits = rolling_edits.dropna()

In [54]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-04-06,0.133333,-0.018351,0.025000
2009-04-07,0.000000,0.000000,0.000000
2009-04-08,0.000000,0.000000,0.000000
2009-04-09,0.000000,0.000000,0.000000
2009-04-10,0.000000,0.000000,0.000000
...,...,...,...
2024-03-31,0.200000,-0.070932,0.083333
2024-04-01,0.200000,-0.070932,0.083333
2024-04-02,0.200000,-0.070932,0.083333
2024-04-03,0.266667,-0.070988,0.100000


In [55]:
rolling_edits.to_csv("wikipedia_edits.csv")